<a href="https://colab.research.google.com/github/mayanlong2020/ColabDesign/blob/main/AfDesign_v1_1_1_%E4%B8%BB%E5%B7%A5%E4%BD%9C%E5%8F%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 1. 环境设置与依赖安装 (v2.2 - 最终健壮版)
# =======================================================================
# 说明 (v2.2 更新日志):
# - 【重大修复】恢复了在v2.0版本中被错误移除的“创建符号链接”步骤。这是导致模块
#   找不到错误的直接原因。
# - 保持了与v1.4版本相同的健壮性：自动清理旧链接、创建安全的非冲突链接。
# - 这是结合了“结构分离”和“环境健壮性”所有成功经验的最终版本。
# =======================================================================

import os
import glob
from google.colab import drive

# --- 1. 挂载 Google Drive ---
print("正在挂载 Google Drive...")
drive.mount('/content/drive', force_remount=True)
print("挂载成功！")

# --- 2. 定义项目核心路径 ---
GDRIVE_PROJECT_PATH = "/content/drive/My Drive/AI_Projects/ColabDesign_v1_1_1"
GDRIVE_REPO_PATH = os.path.join(GDRIVE_PROJECT_PATH, "repository")
GDRIVE_PARAMS_PATH = os.path.join(GDRIVE_PROJECT_PATH, "params")
LOCAL_TAR_PATH = "/content/alphafold_params_2022-12-06.tar"

# --- 3. 智能判断：执行首次安装或快速加载 ---
if not os.path.isdir(GDRIVE_REPO_PATH):
    print("\n检测到首次运行，正在执行一次性完整安装...")

    os.makedirs(GDRIVE_REPO_PATH, exist_ok=True)
    os.makedirs(GDRIVE_PARAMS_PATH, exist_ok=True)

    print("步骤 1/3: 正在克隆 ColabDesign 仓库...")
    os.system(f"git clone https://github.com/sokrypton/ColabDesign.git \"{GDRIVE_REPO_PATH}\"")
    os.system(f"cd \"{GDRIVE_REPO_PATH}\" && git checkout v1.1.1")

    print("\n步骤 2/3: 正在下载 AlphaFold 模型参数...")
    os.system("apt-get install aria2 -qq")
    os.system(f"aria2c -x 16 -s 16 -o {os.path.basename(LOCAL_TAR_PATH)} -d /content https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar")

    if os.path.exists(LOCAL_TAR_PATH):
        print("\n         ...下载成功，正在解压到 Google Drive...")
        os.system(f"tar -xf {LOCAL_TAR_PATH} -C \"{GDRIVE_PARAMS_PATH}\"")
        os.remove(LOCAL_TAR_PATH)
        print("步骤 3/3: 首次安装成功完成！")
    else:
        raise Exception("错误：模型参数下载失败，安装中断。")
else:
    print("\n检测到持久化环境，跳过下载和克隆，执行快速加载。")

# --- 4. 安装 ColabDesign 包 ---
print("\n正在从 Google Drive 安装 ColabDesign 包...")
# 使用 -e (可编辑模式) 并为带空格的路径加上引号
os.system(f"pip install -e \"{GDRIVE_REPO_PATH}\"")
print("\n包安装过程执行完毕。")

# --- 5. 创建符号链接 (用于调试) ---
# 这是修复问题的关键步骤，确保调试的便利性且不与Python导入冲突。
print("正在清理并创建用于调试的源码快捷方式...")

# 【自我修正】主动删除可能存在的、旧的、有问题的符号链接
OLD_LINK_NAME = "colabdesign"
if os.path.islink(OLD_LINK_NAME):
    print(f"检测到旧的、有冲突的符号链接 '{OLD_LINK_NAME}'，正在将其删除...")
    os.remove(OLD_LINK_NAME)

# 创建新的、安全的、无冲突的符号链接
NEW_LINK_NAME = "colabdesign_source_code"
if os.path.islink(NEW_LINK_NAME):
    os.remove(NEW_LINK_NAME)
# 为包含空格的路径加上引号
os.system(f"ln -s \"{GDRIVE_REPO_PATH}\" {NEW_LINK_NAME}")
print(f"快捷方式 '{NEW_LINK_NAME}' 创建成功，指向您 Drive 中的源码目录。")


# --- 6. 导入基础库 (仅限本单元格后续操作所需) ---
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print("\n✅ 持久化环境设置完毕。可以执行下一个单元格来进行蛋白质设计。")

In [ ]:
#@title 2. 配置运行参数并创建沙箱 (v3.2 - 用户体验优化版)
# =======================================================================
# 说明 (v3.2 更新日志):
# - 【用户体验】为所有参数增加了极其详细、生动形象的中文解释，方便领域专家理解。
# - 本单元格是每次新设计的起点，它会自动创建“数据沙箱”并保存所有配置。
# =======================================================================

# --- 导入所需库 ---
import os
import re
import json
import shutil
from datetime import datetime
from google.colab import files
import numpy as np
from colabdesign import mk_afdesign_model, clear_mem
from colabdesign.shared.utils import copy_dict

# --- 1. 创建并设置数据沙箱 ---
# 定义在单元格1中已准备好的全局路径
GDRIVE_PROJECT_PATH = "/content/drive/My Drive/AI_Projects/ColabDesign_v1_1_1"
GDRIVE_PARAMS_PATH = os.path.join(GDRIVE_PROJECT_PATH, "params")
GDRIVE_OUTPUTS_PATH = os.path.join(GDRIVE_PROJECT_PATH, "outputs")

# 创建本次运行的独立沙箱目录
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# 在此定义沙箱路径为全局变量，以便后续单元格使用
run_sandbox_path = os.path.join(GDRIVE_OUTPUTS_PATH, f"run_{timestamp}")
os.makedirs(run_sandbox_path, exist_ok=True)
print(f"✅ 已创建本次运行的数据沙箱:\n{run_sandbox_path}")

# =======================================================================
#                           在这里配置所有参数
# =======================================================================

#@markdown ---
#@markdown ### **1. 靶点信息 (Target Information)**
#@markdown 请提供您希望设计的结合蛋白所要攻击的目标。
pdb = "4N5T" #@param {type:"string"}
#@markdown - **PDB 编码 / UniProt ID**: 您可以把它想象成蛋白质的“三维身份证号”。输入一个4位的PDB ID（如`4N5T`），或一个UniProt ID（程序将从AlphaFold数据库获取预测结构）。
target_chain = "A" #@param {type:"string"}
#@markdown - **目标链 (Target Chain)**: 一个蛋白质PDB文件可能包含多条链（如A, B, C...），请明确指定哪一条是您要作为靶点的链。
target_hotspot = "" #@param {type:"string"}
#@markdown - **靶点上的“热点”区域 (Hotspot)**: (可选) 就像在地图上标记出你希望游客必须参观的几个核心景点。在这里您可以指定靶点上您最感兴趣的氨基酸位置（如`1-10,15,20-25`），程序会重点优化与这些区域的结合。
target_flexible = False #@param {type:"boolean"}
#@markdown - **允许靶点柔性 (Target Flexible)**: (可选) 默认情况下，靶点的骨架是刚性的。勾选此项，则允许靶点自身的结构在结合过程中发生一些微小的变动，以更好地适应我们设计的蛋白。

#@markdown ---
#@markdown ### **2. 结合蛋白信息 (Binder Information)**
#@markdown 请定义您想要从头“创造”的这个结合蛋白的基本属性。
binder_len = 14 #@param {type:"integer"}
#@markdown - **结合蛋白长度 (Binder Length)**: 您希望设计的这个新蛋白质由多少个氨基酸组成。
binder_seq = "" #@param {type:"string"}
#@markdown - **初始序列 (Initial Sequence)**: (可选) 如果您对序列已经有了一些初步想法，可以填在这里。程序将从您的序列开始进行优化，而不是从完全随机的序列开始。
binder_chain = "" #@param {type:"string"}
#@markdown - **已有结合蛋白的链 (Binder Chain)**: (可选) 如果您的PDB文件中已经有一个结合蛋白，您可以在这里指定它的链ID。程序将不再从头设计，而是对这条已有的链进行序列优化，让它结合得更好。

#@markdown ---
#@markdown ### **3. 核心模型配置 (Model Configuration)**
use_multimer = False #@param {type:"boolean"}
#@markdown - **使用多聚体模型 (Use Multimer)**: AlphaFold有两个版本，一个用于单链，一个用于复合物（多聚体）。如果您的靶点和设计的蛋白是两条独立的链，理论上用多聚体模型效果更好，但计算也更慢。对于这个结合任务，非多聚体模型通常已足够。
num_recycles = 0 #@param ["0", "1", "3", "6"] {type:"raw"}
#@markdown - **计算循环次数 (Recycles)**: 就像画家反复审视和修改自己的画作。增加循环次数可以让AlphaFold对结构进行更精细的优化和预测，但也会显著增加计算时间。对于设计任务，0或1通常是效率和效果的平衡点。
num_models = "2" #@param ["1", "2", "3", "4", "5", "all"]
#@markdown - **使用的模型数量 (Number of Models)**: AlphaFold由5个略有不同的模型集成。使用更多的模型进行评估，结果更可靠、更不容易“偏科”，但计算也更慢。可以理解为让一个“专家委员会”而不是单个专家来做决策。

# --- 后续步骤需要的参数，在此一并配置 ---
#@markdown ---
#@markdown ### **4. 优化器与可视化参数**
#@markdown 这些参数将在后续的计算和可视化单元格中使用。
#@markdown #### **优化器策略 (Optimizer Strategy)**
optimizer = "pssm_semigreedy" #@param ["pssm_semigreedy", "3stage", "semigreedy", "pssm", "logits", "soft", "hard"]
#@markdown - `pssm_semigreedy`: **(推荐)** 像一位熟练的工匠，先用梯度下降画出精细的“设计蓝图”(PSSM)，再用半随机的“手工雕刻”来找到最佳序列。
#@markdown - `3stage`: 像一位雕塑家，从一块模糊的“粘土”（连续概率）开始，分三步（logits→soft→hard）逐步将作品雕刻成型。
#@markdown #### **高级梯度下降设置 (Advanced GD Settings)**
GD_method = "sgd" #@param ["adabelief", "adafactor", "adagrad", "adam", "adamw", "fromage", "lamb", "lars", "noisy_sgd", "dpsgd", "radam", "rmsprop", "sgd", "sm3", "yogi"]
#@markdown - **优化算法 (Optimizer Algorithm)**: 选择不同的数学方法来寻找最优解。`sgd` 或 `adam` 是常用的稳健选择。
learning_rate = 0.1 #@param {type:"raw"}
#@markdown - **学习率 (Learning Rate)**: 这好比设计师每一步修改的“力度”。值越大，修改得越快，但可能“用力过猛”错过最佳点；值越小，修改得越精细，但速度慢。
#@markdown #### **3D 可视化选项 (3D Visualization Options)**
color_scheme = "pLDDT" #@param ["chain", "pLDDT", "rainbow", "hydrophobicity"]
#@markdown - **着色方案**: `pLDDT`按可信度着色（蓝高红低），`chain`按链着色，`rainbow`按序列顺序，`hydrophobicity`按疏水性。
show_sidechains = True #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
generate_animation = False #@param {type:"boolean"}

# =======================================================================
#               以下为自动执行部分，无需修改
# =======================================================================

# --- 创建并保存参数文件 ---
all_params = locals()
params_to_save = {
    "靶点信息": {"pdb": pdb, "target_chain": target_chain, "target_hotspot": target_hotspot, "target_flexible": target_flexible},
    "Binder信息": {"binder_len": binder_len, "binder_seq": binder_seq, "binder_chain": binder_chain},
    "模型配置": {"use_multimer": use_multimer, "num_recycles": num_recycles, "num_models": num_models},
    "优化器配置": {"optimizer": optimizer, "GD_method": GD_method, "learning_rate": learning_rate},
    "可视化配置": {"color_scheme":color_scheme, "show_sidechains":show_sidechains, "show_mainchains":show_mainchains, "generate_animation":generate_animation}
}
params_filepath = os.path.join(run_sandbox_path, "run_parameters.json")
with open(params_filepath, "w") as f:
    json.dump(params_to_save, f, indent=4)
print(f"✅ 所有运行参数已保存至: {os.path.basename(params_filepath)}")

# --- 定义健壮的 get_pdb 函数 ---
def get_pdb_and_persist(pdb_code, sandbox_path):
    print(f"正在处理PDB输入: {pdb_code}")
    local_path = os.path.join("/content", os.path.basename(str(pdb_code))) # 在/content下操作
    if os.path.isfile(pdb_code):
        shutil.copy(pdb_code, local_path)
    elif len(pdb_code) == 4:
        os.system(f"wget -q -O {local_path} https://files.rcsb.org/view/{pdb_code}.pdb")
    else:
        os.system(f"wget -q -O {local_path} https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")

    if not os.path.exists(local_path) or os.path.getsize(local_path) == 0:
        raise IOError(f"错误：下载或获取 PDB 文件 {pdb_code} 失败。")
    persistent_path = os.path.join(sandbox_path, f"input_{os.path.basename(local_path)}")
    shutil.copy(local_path, persistent_path)
    print(f"✅ 输入PDB文件已持久化到沙箱: {os.path.basename(persistent_path)}")
    return persistent_path

# --- 模型准备逻辑 ---
binder_seq_cleaned = re.sub("[^A-Z]", "", binder_seq.upper())
binder_len_eff = len(binder_seq_cleaned) if len(binder_seq_cleaned) > 0 else binder_len
binder_seq_final = binder_seq_cleaned if len(binder_seq_cleaned) > 0 else None
binder_chain_final = None if binder_chain == "" else binder_chain
num_models_int = 5 if num_models == "all" else int(num_models)

inputs = {"chain":target_chain, "binder_len":binder_len_eff, "binder_chain":binder_chain_final, "hotspot":target_hotspot,
          "use_multimer":use_multimer, "rm_target_seq":target_flexible}
inputs["pdb_filename"] = get_pdb_and_persist(pdb, run_sandbox_path)

print("\n正在准备并加载 AfDesign 模型...")
clear_mem()
model = mk_afdesign_model(protocol="binder", use_multimer=inputs["use_multimer"],
                          num_recycles=num_recycles, recycle_mode="sample",
                          data_dir=GDRIVE_PARAMS_PATH)
model.prep_inputs(**inputs, ignore_missing=False)

print(f"模型准备完毕。靶点长度: {model._target_len}, 设计长度: {model._binder_len}")
print("\n✅ 第二步：所有参数配置完毕，模型加载成功！")

In [ ]:
#@title 3. 运行 AfDesign 设计流程 (v3.4 - UI修复和最终版)
# =======================================================================
# 说明 (v3.4):
# - 【重大修复】恢复了本单元格的所有UI控件和详细参数解释。
# - 本单元格执行核心的设计计算，并将配置参数和输出PDB都保存到沙箱。
# =======================================================================

# --- 1. 在此配置本步骤的参数 ---
#@markdown ### **优化器策略与高级设置**
optimizer = "pssm_semigreedy" #@param ["pssm_semigreedy", "3stage", "semigreedy", "pssm", "logits", "soft", "hard"]
#@markdown - **优化器策略 (Optimizer Strategy)**: 选择设计的核心算法。`pssm_semigreedy`是兼具速度和效果的推荐选项。
GD_method = "sgd" #@param ["adabelief", "adafactor", "adagrad", "adam", "adamw", "fromage", "lamb", "lars", "noisy_sgd", "dpsgd", "radam", "rmsprop", "sgd", "sm3", "yogi"]
#@markdown - **梯度下降算法 (GD Algorithm)**: 选择不同的数学方法来寻找最优解。`sgd` 或 `adam` 是常用的稳健选择。
learning_rate = 0.01 #@param {type:"raw"}
#@markdown - **学习率 (Learning Rate)**: 设计师每一步修改的“力度”。值越大修改越快，但可能错过最佳点；值越小越精细，但速度慢。
dropout = True #@param {type:"boolean"}
#@markdown - **启用 Dropout (Enable Dropout)**: 一种防止模型“死记硬背”的技术，能增加设计结果的泛化能力。推荐开启。
norm_seq_grad = True #@param {type:"boolean"}
#@markdown - **序列梯度归一化 (Normalize Sequence Gradients)**: 一种稳定训练过程的技术。推荐开启。

# --- 2. 将本单元格的参数保存到沙箱的JSON文件中 ---
params_filepath = os.path.join(run_sandbox_path, "run_parameters.json")
# 读取已有的参数
with open(params_filepath, 'r') as f:
    all_params = json.load(f)
# 更新参数
all_params.update({
    "步骤3_优化器配置": {"optimizer": optimizer, "GD_method": GD_method, "learning_rate": learning_rate,
                     "dropout": dropout, "norm_seq_grad": norm_seq_grad}
})
# 写回文件
with open(params_filepath, 'w') as f:
    json.dump(all_params, f, indent=4)
print(f"✅ 本步骤运行参数已更新并保存至: {params_filepath}")


# --- 3. 执行设计流程 (核心逻辑不变) ---
model.restart(seq=binder_seq_final)
model.set_optimizer(optimizer=GD_method, learning_rate=learning_rate, norm_seq_grad=norm_seq_grad)
models_to_use = model._model_names[:num_models_int]
flags = {"num_recycles":num_recycles, "models":models_to_use, "dropout":dropout}

print(f"\n正在使用 '{optimizer}' 策略开始设计 (这可能需要一些时间)...")

if optimizer == "3stage":
  model.design_3stage(120, 60, 10, **flags)
elif optimizer == "pssm_semigreedy":
  model.design_pssm_semigreedy(120, 32, **flags)
else:
  design_function = getattr(model, f"design_{optimizer}", None)
  if design_function: design_function(120, **flags)
  else: raise ValueError(f"错误：未知的优化器 '{optimizer}'")

# --- 4. 将最终结构保存到沙箱 ---
output_pdb_path = os.path.join(run_sandbox_path, f"{model.protocol}_design.pdb")
model.save_pdb(output_pdb_path)

print(f"\n✅ 第三步：设计流程执行完毕！")
print(f"最终设计的结构已保存到沙箱中: {os.path.basename(output_pdb_path)}")```

### **单元格 4: 结果可视化与分析 (v3.4)**

这个版本与我上次提供的 `v3.3` 相同，是正确的、带有UI的版本。为了确保一致，我再次提供。

```python
#@title 4. 结果可视化与分析 (v3.4)
# =======================================================================
# 说明 (v3.4):
# - 本单元格用于交互式地可视化设计结果。
# - 所有生成的图表（3D视图、动画、热图）都会自动保存到沙箱中。
# =======================================================================
import plotly.express as px
from scipy.special import softmax
from colabdesign.af.alphafold.common import residue_constants

# --- 1. 3D结构可视化与保存 ---
#@markdown ### **3D结构可视化**
#@markdown 您可以在这里交互式地调整视图参数，单元格会自动更新以反映您的更改。
#@markdown ---
color = "pLDDT" #@param ["chain", "pLDDT", "rainbow"]
#@markdown - **着色方案 (Color Scheme)**: `pLDDT`按可信度着色（蓝高红低），`chain`按链着色，`rainbow`按序列顺序。
show_sidechains = True #@param {type:"boolean"}
#@markdown - **显示侧链 (Show Sidechains)**: 勾选以观察氨基酸的侧链结构。
show_mainchains = False #@param {type:"boolean"}
#@markdown - **显示主链条带 (Show Mainchains)**: 勾选以观察整体的折叠走向。
animate = False #@param {type:"boolean"}
#@markdown - **生成轨迹动画 (Generate Animation)**: 勾选以生成展示“设计过程”的动画。**(注意: 可能需要较长时间)**

print("正在生成3D结构视图...")

view = model.plot_pdb(show_sidechains=show_sidechains, show_mainchains=show_mainchains, color=color)
view.show()

view_filepath = os.path.join(run_sandbox_path, "3D_view_static.html")
view.save(view_filepath)
print(f"✅ 静态3D视图已保存到沙箱: {os.path.basename(view_filepath)}")

if animate:
    print("\n正在生成设计轨迹动画 (可能需要一些时间)...")
    animation_view = model.animate()
    animation_filepath = os.path.join(run_sandbox_path, "design_animation.html")
    animation_view.save(animation_filepath)
    print(f"✅ 设计轨迹动画已保存到沙箱: {os.path.basename(animation_filepath)}")


# --- 2. PSSM热图可视化与保存 ---
#@markdown ---
#@markdown ### **氨基酸概率热图 (PSSM)**
#@markdown 如果您使用了生成PSSM的策略，这里会显示每个位置的氨基酸选择概率。
if hasattr(model, '_tmp') and "seq_logits" in model._tmp:
  print("\n正在生成氨基酸概率热图...")
  pssm = softmax(model._tmp["seq_logits"],-1)
  fig = px.imshow(pssm.mean(0).T,
                  labels=dict(x="氨基酸位置", y="氨基酸类型", color="概率"),
                  y=residue_constants.restypes, zmin=0, zmax=1,
                  template="simple_white", title="序列各位置的氨基酸概率 (PSSM)")
  fig.update_xaxes(side="top")
  fig.show()

  pssm_filepath = os.path.join(run_sandbox_path, "pssm_heatmap.html")
  fig.write_html(pssm_filepath)
  print(f"✅ PSSM热图已保存到沙箱: {os.path.basename(pssm_filepath)}")
else:
  print("\n提示: 本次运行未使用生成PSSM的优化器，因此无法显示热图。")

print("\n✅ 第四步：可视化分析执行完毕。")

In [ ]:
#@title 4. 结果可视化与分析 (v3.3 - UI修复和最终版)
# =======================================================================
# 说明 (v3.3 更新日志):
# - 【重大修复】恢复了在v3.2中被错误删除的所有UI控件和详细参数解释。
# - 保持了将所有可视化结果（3D视图、动画、热图）自动保存到沙箱的功能。
# - 这是针对可视化模块的最终修复版本，兼具用户体验和功能完整性。
# =======================================================================
import plotly.express as px
from scipy.special import softmax
from colabdesign.af.alphafold.common import residue_constants

# --- 1. 3D结构可视化与保存 ---
#@markdown ### **3D结构可视化**
#@markdown 您可以在这里交互式地调整视图参数，单元格会自动更新以反映您的更改。
#@markdown ---
color = "pLDDT" #@param ["chain", "pLDDT", "rainbow"]
#@markdown - **着色方案 (Color Scheme)**:
#@markdown  - `pLDDT`: **(推荐)** 按预测的可信度着色。蓝色表示高可信度区域，红色表示低可信度区域。是我们评估设计质量的核心指标。
#@markdown  - `chain`: 按不同的蛋白质链着色，方便地区分靶点和我们设计的结合蛋白。
#@markdown  - `rainbow`: 按氨基酸在序列中的前后顺序，像彩虹一样着色。
show_sidechains = True #@param {type:"boolean"}
#@markdown - **显示侧链 (Show Sidechains)**: 勾选此项可以清晰地看到每个氨基酸的侧链结构，对于分析关键的相互作用位点非常重要。
show_mainchains = False #@param {type:"boolean"}
#@markdown - **显示主链条带 (Show Mainchains)**: 勾选此项会用平滑的条带图来表示蛋白质的骨架，有助于观察整体的折叠走向（如α螺旋和β折叠）。
animate = False #@param {type:"boolean"}
#@markdown - **生成轨迹动画 (Generate Animation)**: 如果优化器在设计过程中保存了多个中间状态，勾选此项可以生成一个展示“设计过程”的动画。**(注意: 可能需要较长时间)**

print("正在生成3D结构视图...")

# 根据UI选择的参数来调用绘图函数
view = model.plot_pdb(show_sidechains=show_sidechains,
                       show_mainchains=show_mainchains,
                       color=color)
view.show()

# 将当前视图保存为HTML文件
view_filepath = os.path.join(run_sandbox_path, "3D_view_static.html")
view.save(view_filepath)
print(f"✅ 静态3D视图已保存到沙箱: {os.path.basename(view_filepath)}")

# 如果用户选择生成动画，则执行并保存
if animate:
    print("\n正在生成设计轨迹动画 (可能需要一些时间)...")
    # model.animate() 会返回一个可以保存的HTML对象
    animation_view = model.animate()
    animation_filepath = os.path.join(run_sandbox_path, "design_animation.html")
    animation_view.save(animation_filepath)
    print(f"✅ 设计轨迹动画已保存到沙箱: {os.path.basename(animation_filepath)}")


# --- 2. PSSM热图可视化与保存 ---
#@markdown ---
#@markdown ### **氨基酸概率热图 (PSSM)**
#@markdown 如果您在第3步中使用了`pssm_semigreedy`或`pssm`等策略，这里会显示一个热图，告诉您在设计的序列中，每个位置上选择各种氨基酸的“倾向性”或概率。
if hasattr(model, '_tmp') and "seq_logits" in model._tmp:
  print("\n正在生成氨基酸概率热图...")
  pssm = softmax(model._tmp["seq_logits"],-1)
  fig = px.imshow(pssm.mean(0).T,
                  labels=dict(x="氨基酸位置 (Position)", y="氨基酸类型 (Amino Acid)", color="概率 (Probability)"),
                  y=residue_constants.restypes, zmin=0, zmax=1,
                  template="simple_white", title="序列各位置的氨基酸概率 (PSSM)")
  fig.update_xaxes(side="top")
  fig.show()

  # 保存热图为HTML文件
  pssm_filepath = os.path.join(run_sandbox_path, "pssm_heatmap.html")
  fig.write_html(pssm_filepath)
  print(f"✅ PSSM热图已保存到沙箱: {os.path.basename(pssm_filepath)}")
else:
  print("\n提示: 本次运行未使用生成PSSM的优化器，因此无法显示热图。")

print("\n✅ 第四步：可视化分析执行完毕。")

In [ ]:
#@title 5. 导出并下载所有结果 (v3.4)
# =======================================================================
# 说明 (v3.4):
# - 本单元格将本次沙箱内的所有关键产出物（包括所有图表）打包成一个 .zip 文件。
# =======================================================================
import zipfile

print(f"正在从沙箱目录导出最终结果:\n{run_sandbox_path}")

# --- 1. 将最终序列保存到沙箱 ---
final_sequence_str = model.get_seqs()[0]
seq_filepath = os.path.join(run_sandbox_path, f"{model.protocol}_sequence.txt")
with open(seq_filepath, "w") as f: f.write(final_sequence_str)
print(f"✅ 序列已保存到: {os.path.basename(seq_filepath)}")

# --- 2. 将详细日志保存到沙箱 ---
log_data = model.get_loss()
log_filepath = os.path.join(run_sandbox_path, f"{model.protocol}_log.json")
with open(log_filepath, "w") as f: json.dump(log_data, f, indent=4, default=lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
print(f"✅ 详细日志已保存到: {os.path.basename(log_filepath)}")

# --- 3. 将所有结果打包成一个 ZIP 文件 ---
zip_filepath = os.path.join(run_sandbox_path, f"AfDesign_results_{timestamp}.zip")
print(f"\n正在将沙箱中的所有文件打包到: {os.path.basename(zip_filepath)}")
with zipfile.ZipFile(zip_filepath, 'w') as zipf:
    for root, dirs, files in os.walk(run_sandbox_path):
        for file in files:
            if not file.endswith('.zip'):
                full_path = os.path.join(root, file)
                zipf.write(full_path, arcname=file)
                print(f"  - 已添加: {file}")

# --- 4. 提供下载选项 ---
#@markdown ---
#@markdown ### **是否下载结果包？**
download_zip = True #@param {type:"boolean"}
if download_zip:
    print("\n准备下载结果包...")
    files.download(zip_filepath)
else:
    print("\n已跳过下载。所有结果均已保存在您的Google Drive沙箱目录中。")

print("\n✅ 第五步：结果导出执行完毕。")